# Introduction

NASA's [AVIRIS-NG](https://avirisng.jpl.nasa.gov) comes as ENVI image format, which might not be amenable to some software and the cloud. This tutorial demonstrates simple conversions to various formats such as [GeoTIFFs](https://www.ogc.org/standards/geotiff), [CoGs](https://www.cogeo.org/), [netCDF](https://www.unidata.ucar.edu/software/netcdf/conventions.html), and [zarr](https://zarr.readthedocs.io/en/stable/).

## Sample file
We will use one Level 2 reflectance file `ang20170706t180635_corr_v2p9_img`, which can be downloaded from [this portal](https://avirisng.jpl.nasa.gov/dataportal/). The `corr` in the file name means that this is a orthocorrected, scaled reflectance image file. The `v2p9` is a version marker. For more information about the file name, read the [filenaming convention](https://avirisng.jpl.nasa.gov/dataportal/ANG_L1B_L2_Data_Product_Readme_v02.txt).

## Converting to netCDF

In [1]:
import rioxarray
avirisng_f = 'ang20170706t180635_corr_v2p9_img'
avirisng = rioxarray.open_rasterio(avirisng_f, driver='ENVI', parse_coordinates=True)
avirisng.attrs['long_name'] = 'orthocorrected scaled reflectance'

Now, the `avirisng` variables contains all bands and attribute information as xarray. The content of the `avirisng` variable is printed below.

In [2]:
avirisng

<xarray.DataArray (band: 425, y: 6454, x: 697)>
[1911836150 values with dtype=float32]
Coordinates:
    wavelength   (band) float64 376.9 381.9 386.9 ... 2.496e+03 2.501e+03
  * band         (band) int64 1 2 3 4 5 6 7 8 ... 419 420 421 422 423 424 425
    xc           (y, x) float64 4.644e+05 4.644e+05 ... 4.323e+05 4.323e+05
    yc           (y, x) float64 7.181e+06 7.181e+06 ... 7.173e+06 7.173e+06
    spatial_ref  int64 0
Dimensions without coordinates: y, x
Attributes:
    wavelength:               376.86
    wavelength_units:         Nanometers
    _FillValue:               -9999.0
    scale_factor:             1.0
    add_offset:               0.0
    long_name:                orthocorrected scaled reflectance
    bad_pixel_map:            /home/winstono/isat-ang2017/ang/cal/data/ANGv5_bad
    bands:                    425
    bbl:                       0.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 ...
    byte_order:               0
    correction_factors:       1.026544,0.975609,0.969727,1.029656,0.988967,1....
    crosstrack_scatter_file:  /home/winstono/isat-ang2017/ang/cal/data/201701...
    data_ignore_value:        -9999
    data_type:                4
    description:                AVIRIS-NG Measured Radiances in uW nm-1 cm-2 ...
    file_type:                ENVI
    flat_field_file:          /home/winstono/isat-ang2017/ang/cal/data/201703...
    fwhm:                     [5.48 5.49 5.49 5.5  5.5  5.51 5.51 5.51 5.52 5...
    header_offset:            0
    interleave:               bil
    lines:                    6454
    radiance_version:         v2.0
    rcc_file:                 /home/winstono/isat-ang2017/ang/cal/data/201705...
    samples:                  697
    smoothing_factors:         1.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 ...
    spectral_scatter_file:    /home/winstono/isat-ang2017/ang/cal/data/201701...
    wavelength_file:          /home/winstono/isat-ang2017/ang/cal/data/201703...

We will add a longitude and latitude coordinates as well.

In [3]:
from pyproj import Transformer
import xarray as xr

transformer = Transformer.from_crs(avirisng.rio.crs, "epsg:4326", always_xy=True) 
lon1, lat1 = transformer.transform(avirisng.xc, avirisng.yc)
avirisng = avirisng.assign_coords(lon=xr.DataArray(lon1, dims=avirisng.xc.dims))
avirisng = avirisng.assign_coords(lat=xr.DataArray(lat1, dims=avirisng.yc.dims))
avirisng = avirisng.drop_vars('band')
avirisng

<xarray.DataArray (band: 425, y: 6454, x: 697)>
[1911836150 values with dtype=float32]
Coordinates:
    wavelength   (band) float64 376.9 381.9 386.9 ... 2.496e+03 2.501e+03
    xc           (y, x) float64 4.644e+05 4.644e+05 ... 4.323e+05 4.323e+05
    yc           (y, x) float64 7.181e+06 7.181e+06 ... 7.173e+06 7.173e+06
    spatial_ref  int64 0
    lon          (y, x) float64 -147.7 -147.7 -147.7 ... -148.4 -148.4 -148.4
    lat          (y, x) float64 64.75 64.75 64.75 64.75 ... 64.67 64.67 64.67
Dimensions without coordinates: band, y, x
Attributes:
    wavelength:               376.86
    wavelength_units:         Nanometers
    _FillValue:               -9999.0
    scale_factor:             1.0
    add_offset:               0.0
    long_name:                orthocorrected scaled reflectance
    bad_pixel_map:            /home/winstono/isat-ang2017/ang/cal/data/ANGv5_bad
    bands:                    425
    bbl:                       0.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 ...
    byte_order:               0
    correction_factors:       1.026544,0.975609,0.969727,1.029656,0.988967,1....
    crosstrack_scatter_file:  /home/winstono/isat-ang2017/ang/cal/data/201701...
    data_ignore_value:        -9999
    data_type:                4
    description:                AVIRIS-NG Measured Radiances in uW nm-1 cm-2 ...
    file_type:                ENVI
    flat_field_file:          /home/winstono/isat-ang2017/ang/cal/data/201703...
    fwhm:                     [5.48 5.49 5.49 5.5  5.5  5.51 5.51 5.51 5.52 5...
    header_offset:            0
    interleave:               bil
    lines:                    6454
    radiance_version:         v2.0
    rcc_file:                 /home/winstono/isat-ang2017/ang/cal/data/201705...
    samples:                  697
    smoothing_factors:         1.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 ...
    spectral_scatter_file:    /home/winstono/isat-ang2017/ang/cal/data/201701...
    wavelength_file:          /home/winstono/isat-ang2017/ang/cal/data/201703...

We will first give a name ("corr") to the dataarray and then convert to a netCDF4 file.

In [4]:
avirsng_ds=avirisng.to_dataset(name='corr')
avirsng_ds

<xarray.Dataset>
Dimensions:      (band: 425, y: 6454, x: 697)
Coordinates:
    wavelength   (band) float64 376.9 381.9 386.9 ... 2.496e+03 2.501e+03
    xc           (y, x) float64 4.644e+05 4.644e+05 ... 4.323e+05 4.323e+05
    yc           (y, x) float64 7.181e+06 7.181e+06 ... 7.173e+06 7.173e+06
    spatial_ref  int64 0
    lon          (y, x) float64 -147.7 -147.7 -147.7 ... -148.4 -148.4 -148.4
    lat          (y, x) float64 64.75 64.75 64.75 64.75 ... 64.67 64.67 64.67
Dimensions without coordinates: band, y, x
Data variables:
    corr         (band, y, x) float32 ...

In [ ]:
avirsng_ds.to_netcdf(avirisng_f + '.nc')